In [30]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sys
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
eps = np.finfo(float).eps

In [31]:
#reading the data
df=pd.read_csv('data.csv')

In [32]:
categorial_attributes= ['Education','Securities_account','CD','internet_banking','credit_card']
numerical_attributes= ['Age','Experience','Annual_income','Family_size','Spending_month','Mortrage']
df=df.drop(['ID','Zipcode'],axis=1)

In [33]:
#spliting the data into test and train
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
X_train=train
Y_test=test['Accept_loan']
X_test=test.drop('Accept_loan',1)

In [34]:
#finding the mean,standarddeviation and normal value
def standarddeviation(column):
    m = mean(column)
    var = sum([pow(x-m,2) for x in column])/float(len(column)-1)
    return math.sqrt(var)
def mean(column):
    return sum(column)/float(len(column))
def normalvalue(x, m, var):
    e = math.exp(-(math.pow(x-m,2)/(2*math.pow(var,2))))
    return (1/(math.sqrt(2*math.pi)*var))*e

In [35]:
def numericaldataprocessing():
    numerical= {}
    for i in numerical_attributes:
        numerical[i] = {}
        numerical[i][1] ={}
        numerical[i][0] = {}
        numerical[i][1]['mean'] = mean(list(X_train[i].iloc[np.where(X_train['Accept_loan']==1.0)]))
        numerical[i][1]['standard'] =  standarddeviation(list(X_train[i].iloc[np.where(X_train['Accept_loan']==1.0)]))
        numerical[i][0]['mean'] = mean(list(X_train[i].iloc[np.where(X_train['Accept_loan']==0.0)]))
        numerical[i][0]['standard'] =  standarddeviation(list(X_train[i].iloc[np.where(X_train['Accept_loan']==0.0)]))
    return numerical

In [36]:
def categoricaldataprocessing():
    categorical = {}
    for i in categorial_attributes:
        categorical[i] = {}
        for j in set(X_train[i]):
            categorical[i][j] = {}
            py = len(np.where((X_train['Accept_loan']==1.0) & (X_train[i]==j))[0])/float(len(np.where(X_train['Accept_loan']==1.0)[0]))
            pn = len(np.where((X_train['Accept_loan']==0.0) & (X_train[i]==j))[0])/float(len(np.where(X_train['Accept_loan']==0.0)[0]))
            categorical[i][j][1] = py
            categorical[i][j][0] = pn
    return categorical        

In [37]:
def predict(X_test,Y_test):
    num_data = numericaldataprocessing()
    cat_data = categoricaldataprocessing()
    Y_pred = []
    test = ['Annual_income','Family_size','Spending_month','Education','Age','Experience','Mortrage','Securities_account','CD','internet_banking','credit_card']
    for i in range(len(X_test)):
        a1 = 1
        a2 = 1
        for j in test:
            if j in numerical_attributes:
                a1=a1*normalvalue(X_test.iloc[i][j],num_data[j][1]['mean'],num_data[j][1]['standard'])
                a2=a2*normalvalue(X_test.iloc[i][j],num_data[j][0]['mean'],num_data[j][0]['standard'])
            else:
                a1=a1*cat_data[j][X_test.iloc[i][j]][1]
                a2=a2*cat_data[j][X_test.iloc[i][j]][0]
        
        a1=a1*len(np.where(X_train['Accept_loan']==1.0)[0])/float(len(X_train))
        a2=a2*len(np.where(X_train['Accept_loan']==0.0)[0])/float(len(X_train)) 
        one = a1/float(a2+a1)
        zero = a2/float(a2+a1)
        if(one > zero):
            Y_pred.append(1)
        else:
            Y_pred.append(0)
    print "The below details are from model developed:-"
    print "Accuracy score:-"
    print accuracy_score(Y_test, Y_pred)
    print "Confusion Matrix:-"
    print confusion_matrix(Y_test,Y_pred)
    print "Classification Report:-"
    print classification_report(Y_test,Y_pred)

In [38]:
predict(X_test,Y_test)

The below details are from model developed:-
Accuracy score:-
0.9149659863945578
Confusion Matrix:-
[[753  45]
 [ 30  54]]
Classification Report:-
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       798
           1       0.55      0.64      0.59        84

   micro avg       0.91      0.91      0.91       882
   macro avg       0.75      0.79      0.77       882
weighted avg       0.92      0.91      0.92       882



In [39]:
# for testing data against the sample data
filename=sys.argv[1]
df2= pd.read_csv(filename)
Y_test1=df2['Accept_loan']
X_test1=df2.drop('Accept_loan',axis=1)
predict(X_test1,Y_test1)

The below details are from model developed:-
Accuracy score:-
0.9079795510113359
Confusion Matrix:-
[[3811  254]
 [ 160  274]]
Classification Report:-
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      4065
           1       0.52      0.63      0.57       434

   micro avg       0.91      0.91      0.91      4499
   macro avg       0.74      0.78      0.76      4499
weighted avg       0.92      0.91      0.91      4499

